In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pandas as pd
import math
from datetime import datetime

In [8]:
class ML:
    def __init__(self):
        # Leer el archivo CSV
        #self.data = np.loadtxt('knapsack_data.csv', delimiter=',', skiprows=1)
        self.data = pd.read_csv('mitbih_test.csv', header= None)
        cantidad_columnas = len(self.data.columns)
        self.fitness_history = []
        #print("Cantidad de columnas: ", cantidad_columnas)

    def loadData(self, selected_features):
        # Usa las caracteristicas dadas por la MH
        ## Devuelve el conjunto de datos filtrado basado en las características seleccionadas.
        all_features = list(self.data.columns)
        print("Cantidad de columnas: ", len(all_features))
        selected_column_names = [all_features[i] for i in range(len(selected_features)) if selected_features[i] == 1]
        return self.data[selected_column_names].values, self.data.iloc[:, -1].values.astype(int)
    
    def obtenerCantidadColumnas(self):
        # Devuelve la cantidad total de columnas en el conjunto de datos.
        return len(self.data.columns)
    
# Función de fitness para la selección de características
    def fitness(self, features, X, y):
        # Seleccionamos las características de acuerdo a la solución (features)
        ## Evalúa una solución basada en la precisión de un clasificador k-NN.
        selected_features = [i for i, val in enumerate(features) if val == 1]
        X_selected = X[:, selected_features]

        # Dividimos en conjunto de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

        # Evaluamos con KNN
        knn_clf = KNeighborsClassifier()
        knn_clf.fit(X_train, y_train)
        knn_predictions = knn_clf.predict(X_test)
        knn_accuracy = accuracy_score(y_test, knn_predictions)

        # Evaluamos con Decision Tree
        dt_clf = DecisionTreeClassifier()
        dt_clf.fit(X_train, y_train)
        dt_predictions = dt_clf.predict(X_test)
        dt_accuracy = accuracy_score(y_test, dt_predictions)

        # Evaluamos con Random Forest
        rf_clf = RandomForestClassifier()
        rf_clf.fit(X_train, y_train)
        rf_predictions = rf_clf.predict(X_test)
        rf_score = accuracy_score(y_test, rf_predictions)

        # Evaluamos con SVM
        svm_clf = SVC()
        svm_clf.fit(X_train, y_train)
        svm_predictions = svm_clf.predict(X_test)
        svm_score = accuracy_score(y_test, svm_predictions)

        # Usamos la precisión como medida de fitness
        return (knn_accuracy, dt_accuracy, rf_score, svm_score)

    def levy_flight(self, beta):
        ## Genera un paso basado en una distribución de Lévy.
        # Factor de escala para el paso
        sigma = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) / 
                (math.gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)

        u = np.random.normal(0, sigma, 1)
        v = np.random.normal(0, 1, 1)
        step = u / abs(v)**(1 / beta)
    
        return step

    def generate_nest(self, num_features):
        ## Genera un nido aleatorio (una solución binaria) basado en la cantidad de características.
        return np.random.choice([0, 1], size=num_features)
    
    def init_q_table(self, num_nests, num_actions=2):
        #Inicializamos Q-table con ceros. Las acciones son 0: No actualizar 1:Actualizar
        return np.zeros((num_nests, num_actions))
    
    def choose_action(self, state, Q, epsilon=0.1):
        # Selecciona una acción basada en una política epsilon-greedy.
        if np.random.uniform(0,1) < epsilon:
            return np.random.choice([0,1])
        else:
            return np.argmax(Q[state, :])
         

    def CSA(self, X, y, pa, max_iter, epsilon, beta, alpha, gamma, num_nests=None):
        # Algoritmo de Cuckoo Search con aprendizaje por refuerzo.
        num_features = X.shape[1]

        if num_nests is None:
            num_nests = len(self.data.columns)
        
        Q = self.init_q_table(num_nests)
            
        # Paso 1: Inicializar nidos
        nests = [self.generate_nest(num_features) for _ in range(num_nests)]
        fitness_values = [self.fitness(nest, X, y) for nest in nests]

        # Inicializar el mejor nido y su puntuación.
        best_nest = None
        best_knn_score = -1
        best_dt_score = -1
        best_rf_score = -1
        best_svm_score = -1

        for iter in range(max_iter):
            print("Iteracion: ", iter+1)
            # Paso 2: Calcular el fitness de cada nido con cada uno de los clasificadores

            knn_scores = [score[0] for score in fitness_values]
            dt_scores = [score[1] for score in fitness_values]
            rf_scores = [score[2] for score in fitness_values]
            svm_scores = [score[3] for score in fitness_values]
            
            #scores = fitness_values.copy()
            
            # Agregar el fitness en el listado de los mejores fitness
            self.fitness_history.append((max(knn_scores), max(dt_scores), max(rf_scores), max(svm_scores)))

            # Paso 3: Encuentra el mejor nido o identificar el nido con la mejor puntuación.
            current_best_nest = nests[np.argmax(knn_scores)]
            current_best_knn_score = max(knn_scores)
            current_best_dt_score = max(dt_scores)
            current_best_rf_score = max(rf_scores)
            current_best_svm_score = max(svm_scores)

            if current_best_knn_score > best_knn_score:
                best_knn_score = current_best_knn_score
                best_nest = current_best_nest

            if current_best_dt_score > best_dt_score:
                best_dt_score = current_best_dt_score
                best_nest = current_best_nest

            if current_best_rf_score > best_rf_score:
                best_rf_score = current_best_rf_score
                best_nest = current_best_nest

            if current_best_svm_score > best_svm_score:
                best_svm_score = current_best_svm_score
                best_nest = current_best_nest

            # Paso 4: Actualizar nidos basado en Lévy flight y aprendizaje Q-learning.
            for i in range(num_nests):
                action = self.choose_action(i, Q)
                
                #Guardamos el score original de cada uno de los clasifiicadores
                original_knn_score = knn_scores[i]
                original_dt_score = dt_scores[i]
                original_rf_score = rf_scores[i]
                original_svm_score = svm_scores[i]
                
                # Si la acción es actualizar el nido.
                if action == 1:
                    if np.random.rand() > pa:
                        # Aplicar Levy Flight para actualizar el nido.
                        #nests[i] = nests[i] + self.levy_flight() * (best_nest - nests[i])
                        #Utiliza el valor absoluto del vuelo de Lévy como una probabilidad p
                        for j in range(num_features):
                            if abs(self.levy_flight(beta)) > 0.5:
                                nests[i][j] = 1 - nests[i][j]
                        #nests[i] = nests[i] + (np.random.rand(num_features) > 0.5) * (best_nest - nests[i])

                        # Reemplazar algunas características aleatoriamente para diversificación
                        ## Diversificación: alterar una característica al azar.
                        nests[i][np.random.choice(num_features)] = 1 - nests[i][np.random.choice(num_features)]
                        fitness_values[i] = self.fitness(nests[i], X, y)
                else:
                    #No se hace nada, no se actualiza el nido
                    pass

                
                # Actualizar la Q-table basado en la recompensa.
                new_knn_score, new_dt_score, new_rf_score, new_svm_score = fitness_values[i]
                #Recompensa: Variacion entre el fitness de la iteracion actual con el fitness de la iteracion pasada
                reward_knn = new_knn_score - original_knn_score
                reward_dt = new_dt_score - original_dt_score
                reward_rf = new_rf_score - original_rf_score
                reward_svm = new_svm_score - original_svm_score
                reward_avg = (reward_knn + reward_dt + reward_rf + reward_svm) / 4
                #Q[i, action] = Q[i, action] + 0.1 * (reward + 0.9 * np.max(Q[i, :]) - Q[i, action])
                next_max_Q = np.max(Q[i, :])  # Estimación del valor Q máximo para el siguiente estado
                Q[i, action] = Q[i, action] + alpha * (reward_avg + gamma * next_max_Q - Q[i, action])
        
        # Devuelve el mejor nido, su puntuación y la Q-table final.
        return best_nest, (best_knn_score, best_dt_score, best_rf_score, best_svm_score), Q

    def get_fitness_history(self):
        return self.fitness_history

In [9]:
ml_instance = ML()

X, y = ml_instance.loadData([1 for _ in range(ml_instance.obtenerCantidadColumnas()-1)]) # Cargar todas las características inicialmente

#Config: {'pa': 0.5, 'max_iter': 10, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6}
config = "'pa': 0.5, 'max_iter': 100, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6"
print("CONFIGURACION: ", config)
best_features, best_accuracy, q_table = ml_instance.CSA(X, y, pa=0.5, max_iter=100, epsilon=0.16, beta=1.4, alpha=0.1, gamma=0.6)
fitness_evolution = ml_instance.get_fitness_history()
knn_accuracy = best_accuracy[0]
dt_accuracy = best_accuracy[1]
rf_accuracy = best_accuracy[2]
svm_accuracy = best_accuracy[3]

knn_history = [item[0] for item in fitness_evolution]
dt_history = [item[1] for item in fitness_evolution]
rf_history = [item[2] for item in fitness_evolution]
svm_history = [item[3] for item in fitness_evolution]

#Escribir los resultados en los archivos correspondientes
with open("RESULTADOS-QLCSA.txt", "a") as file:
    file.write("Fecha y hora: " + str(datetime.now()) + "\n")
    file.write("Parametros utilizados: " + str(config) + "\n")
    file.write("Mejores características: " + str(best_features) + "\n")
    file.write("Fitness (KNN): " + str(knn_accuracy) + "\n")
    file.write("Fitness (Decision Tree): " + str(dt_accuracy) + "\n")
    file.write("Fitness (Random Forest): " + str(rf_accuracy) + "\n")
    file.write("Fitness (SVM): " + str(svm_accuracy) + "\n")
    file.write("-------------------------------------------------------------------------\n\n")

with open("QTABLE-QLCSA.txt", "a") as file:
    file.write(str(datetime.now()))
    file.write("\n" + " Q table: " + str(q_table) + "\n\n")
    file.write("-------------------------------------------------------------------------\n\n")
    
with open("FITNESSEVOLUTION-QLCSA.txt", "a") as file:
    file.write(str(datetime.now()))
    file.write("\n"+ "KNN Fitness Evolution: " + str(knn_history) + "\n\n")
    file.write("\n"+ "Decision Tree Fitness Evolution: " + str(dt_history) + "\n\n")
    file.write("\n"+ "Random Forest Fitness Evolution: " + str(rf_history) + "\n\n")
    file.write("\n"+ "SVM Fitness Evolution: " + str(svm_history) + "\n\n")
    file.write("-------------------------------------------------------------------------\n\n")

Cantidad de columnas:  188
CONFIGURACION:  'pa': 0.5, 'max_iter': 100, 'epsilon': 0.16, 'beta': 1.4, 'alpha': 0.1, 'gamma': 0.6
Iteracion:  1
Iteracion:  2
Iteracion:  3
Iteracion:  4
Iteracion:  5
Iteracion:  6
Iteracion:  7
Iteracion:  8
Iteracion:  9
Iteracion:  10
Iteracion:  11
Iteracion:  12
Iteracion:  13
Iteracion:  14
Iteracion:  15
Iteracion:  16
Iteracion:  17
Iteracion:  18
Iteracion:  19
Iteracion:  20
Iteracion:  21
Iteracion:  22
Iteracion:  23
Iteracion:  24
Iteracion:  25
Iteracion:  26
Iteracion:  27
Iteracion:  28
Iteracion:  29
Iteracion:  30
Iteracion:  31
Iteracion:  32
Iteracion:  33
Iteracion:  34
Iteracion:  35
Iteracion:  36
Iteracion:  37
Iteracion:  38
Iteracion:  39
Iteracion:  40
Iteracion:  41
Iteracion:  42
Iteracion:  43
Iteracion:  44
Iteracion:  45
Iteracion:  46
Iteracion:  47
Iteracion:  48
Iteracion:  49
Iteracion:  50
Iteracion:  51
Iteracion:  52
Iteracion:  53
Iteracion:  54
Iteracion:  55
Iteracion:  56
Iteracion:  57
Iteracion:  58
Iteracion: 